# RosBag python class to process bag files 

- Python 3.6 environment
- This notebook runs on a ROS kinetic environment
- Notebook with ros runs via: pip install jupyros
- rosbag was installed using pip3

In [21]:
# TODO: Discuss with divin if bag_info and bag content are necessary
# TODO: convert to csv 
# TODO: use pandas to plot

## Import libraries

In [6]:
# jupyter kernel
import sys
print(sys.version)

3.6.9 (default, Jul  3 2019, 15:36:16) 
[GCC 5.4.0 20160609]


In [63]:
import rosbag
import time
import glob
import rospy
import pandas as pd 
import numpy as np
import rosbag_pandas

In [51]:
class RosBag():
    """
    RosBag object containing methods to:
    Read, pre-process and plot data given a ROS bag file
    
    Input: bag_name
        string: name of bag file in directory
    """
    
    def __init__(self, bag_name):
        self.bag = rosbag.Bag(bag_name)
    
    def convert_time(self, tm):
        return time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime(tm))
    
    def read_message(msg):
        headers = msg.__slots__
        for i in headers:
            print(i + ' : ' + str(getattr(msg, i)))
        
    def bag_metadata(self):
        print('--------------------------------------------------------------------')
        print("RosBag meta data:")
        print(self.bag)

    def bag_info(self):
        Start_Time = self.convert_time(self.bag.get_start_time())
        End_Time = self.convert_time(self.bag.get_end_time())
        Messages = self.bag.get_type_and_topic_info()[0].keys()
        self.Topics = self.bag.get_type_and_topic_info()[1].keys()
        self.msgs = [self.bag.get_type_and_topic_info()[1].get(i) for i in self.bag.get_type_and_topic_info()[1].keys()]
        
        print('Start Time : ' + str(Start_Time))
        print('End Time : ' + str(End_Time))
        print('Topics : ' + str(self.Topics))
        print('Messages : ' + str(Messages))
        
        print('--------------------------------------------------------------------')
        # TODO: printing is not convenient for long files
        print('Topics and Messages :')
        for idx, j in enumerate(self.Topics):
            print('\n')
            print('Topic : ' + str(j))
            print('Message : ' + str(self.msgs[idx].msg_type))
            print('Count : ' + str(self.msgs[idx].message_count))
            print('Connections : ' + str(self.msgs[idx].connections))
            print('Frequency : ' + str(round(self.msgs[idx].frequency, 1)) + ' Hz')
            
        # self.bag.close()
    
    def bag_to_pandas(self):
        # Get topics and messages
        self.Topics = self.bag.get_type_and_topic_info()[1].keys()
        self.msgs = [self.bag.get_type_and_topic_info()[1].get(i) for i in self.bag.get_type_and_topic_info()[1].keys()]
        
        # Create Pandas dataframe
        df = pd.DataFrame(columns=self.Topics)
        data = dict()
        data[' '] = ['Message', 'Count', 'Connections', 'Frequency']
        for idx, j in enumerate(self.Topics):
            data[j] = [self.msgs[idx].msg_type, self.msgs[idx].message_count, 
                       self.msgs[idx].connections, round(self.msgs[idx].frequency, 1)]
        
        df1 = pd.DataFrame(data)
        df = df.append(df1, ignore_index = True) 
        return df

## Read bagfiles in folder


In [52]:
bag_names = glob.glob("bagfiles/*.bag")
print(bag_names[0])

bagfiles/turtle_simulation.bag


## Instantiate ros_bag object

In [53]:
ros_bag_turtle_sim = RosBag(bag_names[0])

In [54]:
ros_bag_turtle_sim.bag_metadata()

--------------------------------------------------------------------
RosBag meta data:
path:        bagfiles/turtle_simulation.bag
version:     2.0
duration:    2:23s (143s)
start:       Oct 24 2019 13:52:05.93 (1571917925.93)
end:         Oct 24 2019 13:54:29.80 (1571918069.80)
size:        1.2 MB
messages:    18039
compression: none [2/2 chunks]
types:       geometry_msgs/Twist [9f195f881246fdfa2798d1d3eebca84a]
             rosgraph_msgs/Log   [acffd30cd6b6de30f120938c17c593fb]
             turtlesim/Color     [353891e354491c51aabe32df673fb446]
             turtlesim/Pose      [863b248d5016ca62ea2e895ae5265cf9]
topics:      /rosout                    4 msgs @ 54.5 kHz : rosgraph_msgs/Log   (2 connections)
             /turtle1/cmd_vel          76 msgs @   4.0 Hz : geometry_msgs/Twist
             /turtle1/color_sensor   8979 msgs @  62.8 Hz : turtlesim/Color    
             /turtle1/pose           8980 msgs @  62.8 Hz : turtlesim/Pose


In [55]:
ros_bag_turtle_sim.bag_info()

Start Time : Thu, 24 Oct 2019 13:52:05 +0000
End Time : Thu, 24 Oct 2019 13:54:29 +0000
Topics : dict_keys(['/rosout', '/turtle1/cmd_vel', '/turtle1/color_sensor', '/turtle1/pose'])
Messages : dict_keys(['geometry_msgs/Twist', 'rosgraph_msgs/Log', 'turtlesim/Color', 'turtlesim/Pose'])
--------------------------------------------------------------------
Topics and Messages :


Topic : /rosout
Message : rosgraph_msgs/Log
Count : 4
Connections : 2
Frequency : 54471.5 Hz


Topic : /turtle1/cmd_vel
Message : geometry_msgs/Twist
Count : 76
Connections : 1
Frequency : 4.0 Hz


Topic : /turtle1/color_sensor
Message : turtlesim/Color
Count : 8979
Connections : 1
Frequency : 62.8 Hz


Topic : /turtle1/pose
Message : turtlesim/Pose
Count : 8980
Connections : 1
Frequency : 62.8 Hz


## Convert to pandas

In [57]:
# dataframe = rosbag_pandas.bag_to_dataframe(bag_names[0])
dataframe_turtlesim = ros_bag_turtle_sim.bag_to_pandas()
dataframe_turtlesim.head()

/home/hackerman/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,,/rosout,/turtle1/cmd_vel,/turtle1/color_sensor,/turtle1/pose
0,Message,rosgraph_msgs/Log,geometry_msgs/Twist,turtlesim/Color,turtlesim/Pose
1,Count,4,76,8979,8980
2,Connections,2,1,1,1
3,Frequency,54471.5,4,62.8,62.8


In [65]:
# dataframe = rosbag_pandas.bag_to_dataframe(bag_names[0])